source API URL : "https://geocoding-api.open-meteo.com/v1/search?name=kovilpatti&count=10&language=en&format=json"

JSON Target File Path : "abfss://working-labs@datalakestorageaccountname.dfs.core.windows.net/bronze/geo-location/
"

In [0]:
geoLocationSourceAPIURL = "https://geocoding-api.open-meteo.com/v1/search?name=kovilpatti&count=10&language=en&format=json"

geoLocationSourceAPIBaseURL = "https://geocoding-api.open-meteo.com/v1/search?name="
geoLocationSourceAPIURLOptions = "&count=10&language=en&format=json"

geoLocationSinkLayerName = 'bronze'
geoLocationStorageAccountName = 'azuredatalakevrs'
geoLocationSinkFolderName = 'geo-location'
geolocationSinkFolderPath = f"abfss://{geoLocationSinkLayerName}@{geoLocationStorageAccountName}.dfs.core.windows.net/{geoLocationSinkFolderName}"


In [0]:
import requests
import json
import pandas as pds


In [0]:
geoLocationAPIResponse = requests.get(geoLocationSourceAPIURL).json()
geoLocationPandasDF = pds.DataFrame(geoLocationAPIResponse)
geoLocationSparkDF = spark.createDataFrame(geoLocationPandasDF)



In [0]:
dailyPricingMarketNamesDF = spark.sql("select Market_Name from pricing_analytics_vrs.gold.reporting_dim_market_gold")

In [0]:
marketNames = [dailyPricingMarketNames["Market_Name"] for dailyPricingMarketNames in dailyPricingMarketNamesDF.collect()]

#print(marketNames)

geoLocationAPIResponseList = []

for marketName in marketNames:
  print(marketName)
  geoLocationSourceAPIURL =f"{geoLocationSourceAPIBaseURL}{marketName}{geoLocationSourceAPIURLOptions}"
  geoLocationAPIResponse = requests.get(geoLocationSourceAPIURL).json()

  if isinstance(geoLocationAPIResponse, dict):
      geoLocationAPIResponseList.append(geoLocationAPIResponse)

print(geoLocationAPIResponse)

geoLocationSparkRDD = sc.parallelize(geoLocationAPIResponseList)

geoLocationSparkDF = spark.read.json(geoLocationSparkRDD)

(geoLocationSparkDF
 .filter("results.admin1 is not null")
   .write
   .mode("overwrite")
   .json(geolocationSinkFolderPath)
)
  

In [0]:
geoLocationBronzeDF = spark.read.json(geolocationSinkFolderPath)
display(geoLocationBronzeDF)